# LLM Basic with Ollama
Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary.

## Install required packages for LLM integration
Run this cell first - you'll see installation progress for each package. After running this, you'll be ready to import and use these packages in your LLM project!

In [ ]:
!pip3 install bs4 ollama requests

## Importing Libraries

In [ ]:
import ollama
import requests

from bs4 import BeautifulSoup
from IPython.display import Markdown, display

## Define constants for Jupyter notebook

In [ ]:
MODEL = "llama3"

## Create website object
This class initializes a `Website` object with a given URL, fetches the website's content, extracts the title, removes irrelevant elements, and stores the remaining text content. This class can be used to analyze and process web pages in a structured manner.

In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, features="html.parser")
        self.title = soup.title.string if soup.title else None

        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator='\n', strip=True)
        

## System Prompt
The `system_prompt` variable is a string that defines the instructions or prompt for an assistant. In this case, the assistant is expected to perform the following tasks:

1. Analyze the contents of a website.
2. Focus on the main content, excluding navigation-related text.
3. Generate a concise summary of the website's content.
4. Format the summary using markdown.

Markdown is a lightweight markup language used for formatting text. It allows the assistant to apply various formatting elements such as headers, emphasis, lists, links, and code blocks using a simple syntax.

The purpose of this system prompt is to guide the assistant in providing a well-structured and readable summary of a website's content, making it easier for users to understand the main points without being overwhelmed by navigation-related information.

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"""You are looking for website titled {website.title}.
                      The contents of this website is as follows in tripple backticks:
                      ```
                      {website.text}
                      ```
                      please provide a short summary of this website in markdown.
                      If it includes news or announcements, then summarize these too."""
    return user_prompt

In [ ]:
def message(website):
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_for(website=website)},
  ]
  return messages

In [ ]:
def summarize(url):
  website = Website(url=url)
  response = ollama.chat(model=MODEL, messages=message(website=website))
  return response['message']['content']

In [ ]:
def display_summary(url):
  summary = summarize(url=url)
  display(Markdown(summary))

In [ ]:
display_summary("https://www.google.com/")